In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

In [ ]:
# book = pd.read_parquet('/kaggle/input/hw1parquets/book.parquet')
book = pd.read_parquet('/kaggle/input/hw1parquets/book.spot.parquet')

book.head(3)

In [ ]:
# trades = pd.read_parquet('/kaggle/input/hw1parquets/trades.parquet')
trades = pd.read_parquet('/kaggle/input/hw1parquets/trades.spot.parquet')

trades.head(3)

trades['remote_ts2'] = trades['remote_ts2'] * 1000000

end_mem = book.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
end_mem = trades.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
float_cols = book.select_dtypes(include='float').columns.tolist()
book[float_cols]=book[float_cols].astype('float16')
print(book.memory_usage(deep=True).sum() / 1024**2)


float_cols = trades.select_dtypes(include='float').columns.tolist()
trades[float_cols]=trades[float_cols].astype('float16')
print(trades.memory_usage(deep=True).sum() / 1024**2)
gc.collect()

**Corrected queue diff**

In [ ]:
# book = pd.read_parquet('/kaggle/input/hw1parquets/book.parquet')

mid_price = (book['asks[0].price'] + book['bids[0].price']) / 2
ask0volDiff = book['asks[0].amount'].diff()
bid0volDiff = book['bids[0].amount'].diff()
ask1volDiff = book['asks[0].amount'] - book['asks[1].amount'].shift()
bid1volDiff = book['bids[0].amount'] - book['bids[1].amount'].shift()

neg_change_mask = mid_price.diff() < 0.0
pos_change_mask = mid_price.diff() > 0.0
no_change_mask = mid_price.diff() == 0.0

book['NewAskDiff'] = ask0volDiff * no_change_mask + ask1volDiff * pos_change_mask
book['NewBidDiff'] = bid0volDiff * no_change_mask + bid1volDiff * neg_change_mask

Мы знаем что если цена изменилась на 1 тик вверх то бест аск это то что было раньше на 1 уровень выше

Мы знаем что если цена изменилась на 1 тик вниз то бест бид это то что было раньше на 1

Если цена не изменилась на 1 тик, то ничего не сдвинулось

Таким образом можно сделать как фичу дифф best ask \ best_bid с поправкой на то что когда цена меняется более чем на 1 тик вниз или вверх то у соответвующей части бука по другому считается дифф

In [ ]:
plt.plot(ask_vols_sum.diff())

In [ ]:
trades.drop(columns=['seq', 'remote_ts',], inplace=True)
trades = trades.sort_values('remote_ts2')
book = book.sort_values('timestamp')

In [ ]:
merged = pd.merge_asof(trades, book[['timestamp', 'asks[0].amount', 'bids[0].amount']], left_on='remote_ts2', right_on='timestamp', direction='forward')
gc.collect()
merged.head()

In [ ]:
merged[['side', 'amount', 'timestamp']].groupby(['timestamp','side'])['amount'].sum()

In [ ]:
# sample = merged.iloc[:200000]
executed = merged[['side', 'amount', 'timestamp']].groupby(['timestamp','side'])['amount'].sum().unstack().fillna(0.).reset_index()
executed = executed.rename(columns={"B": "buy_amount", "S": "sell_amount"})
executed['total_amount'] = executed['buy_amount'] + executed['sell_amount']
executed.head()

In [ ]:
executed.to_parquet('executed.parquet')

## **Load Executed**

In [ ]:
executed = pd.read_parquet('/kaggle/input/hw1parquets/executed.parquet')
executed[['buy_amount', 'sell_amount', 'total_amount']] = executed[['buy_amount', 'sell_amount', 'total_amount']].astype('float16')
book = pd.merge(book[['timestamp','NewAskDiff', 'NewBidDiff', 'asks[0].amount', 'bids[0].amount']], executed, on='timestamp', how='left').fillna(0.)
# book.head(100).tail(20)
gc.collect()
book['dt'] = pd.to_datetime(book['timestamp'])
book.set_index('dt', inplace=True)

In [ ]:
windows = ['500ms', '1500ms', '2500ms', '5000ms']
for window in windows:
    askExecSum = book['buy_amount'].rolling(window).sum()
    bidExecSum = book['sell_amount'].rolling(window).sum()
    # timeFirst = book.resample('5000ms', on='timestamp').first()
    timeFirst = book['timestamp'].rolling(window).min()
    timeLast = book['timestamp']

    book[f'AskExecRate{window}'] = askExecSum / (book['timestamp'] - timeFirst)
    book[f'BidExecRate{window}'] = bidExecSum / (book['timestamp'] - timeFirst)

    book[f'askExecSum{window}'] = askExecSum
    book[f'bidExecSum{window}'] = bidExecSum

    book[f'timeAskExecute{window}'] = book['asks[0].amount'] / book[f'AskExecRate{window}']
    book[f'timeBidExecute{window}'] = book['bids[0].amount'] / book[f'BidExecRate{window}']

book.head(1000).tail(15)

Думаю, что уместно будет кол-во продаж, покупок и их общее количество считать во временном окне, так как видно, что много мест когда в предыдущий момент еще ничего не произошло и тут происходит потеря информации, лучше немного сгладить но получить более полное представление.

Сложно сразу сказать какое окно подойдет лучше так что я выбрал 3 разных.

Далее простая фича, хочу попробовать,вдруг это сможет сработать. Стандартные отклонения за период в окне. Правда мне кажется что более умные фичи, из 2-ой дз будут покрывать вклад этих. 

In [ ]:
book['250msStd'] = book['total_amount'].rolling(window='250ms').std()
book['500msStd'] = book['total_amount'].rolling(window='500ms').std()
book['1000msStd'] = book['total_amount'].rolling(window='1000ms').std()
book['2500msStd'] = book['total_amount'].rolling(window='2500ms').std()

Кажется что теперь мы можем рассматривать колонку amount как объем трейдов совершенных между текущим снимком бука и предыдущим. А значит можем оценить такие крутые величины как cancellation rate, execution rate, arrival rate (потом можно еще немножко упоротья и оценить отдельно по каждой стороне.)

Идеи: 

Для каждого из окон считать частоту исполненных ордеров и частоту arrivals-cancellations.

Считать отклонения \ (частное) частот в маленьком окне от более глобальных

In [ ]:
dt = pd.to_datetime(book['timestamp'])

book['hour'] = dt.dt.hour

def partOfDay(x):
    if x >= 22 or x < 8:
        return 0 #"Night"
    if x < 12:
        return 1 #"Morning"
    if x < 16:
        return 2 #"Afternoon"
    if x < 22:
        return 3 # "Evening"

book['part_of_day'] = book['hour'].apply(lambda x: partOfDay(x))

In [ ]:
plt.plot(book['part_of_day'])

In [ ]:
plt.plot(ams['2500msStd'][::1000])

In [ ]:
gc.collect()

In [ ]:
book.columns

## Ticker features (trying to time windowed vol avgs)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ticker = pd.read_csv('/kaggle/input/hw1parquets/ticker.csv/ticker.csv')
# print(ticker.memory_usage(deep=True).sum() / 1024**2)
ticker.drop(columns=['seq', 'remote_ts2'], inplace=True)
# print(ticker.memory_usage(deep=True).sum() / 1024**2)
# print(ticker.dtypes)
float_cols = ticker.select_dtypes(include='float').columns.tolist()
ticker[float_cols]=ticker[float_cols].astype('float32')
print(ticker.memory_usage(deep=True).sum() / 1024**2)
# print(ticker.dtypes)

**Read target to see feature correlations**

In [ ]:
ticker['spread'] = ticker['ask_price'] - ticker['bid_price']

In [ ]:
plt.plot(spread)

In [ ]:
ticker['amount'] = ticker['bid_amount'] + ticker['ask_amount']

In [ ]:
ticker['dt'] = pd.to_datetime(ticker['local_ts'])
ticker.set_index('dt', inplace=True)

In [ ]:
plt.figure()
plt.plot(10+avgVol6h)
plt.plot(15+avgVol3h)
plt.plot(20+avgVol1h)
plt.plot(25+avgVol15m, alpha=0.5)
plt.plot(30+avgVol5m, alpha=0.25)
plt.plot(35+avgVol1m, alpha=0.1)
plt.show()

In [ ]:
avgVol6h = ticker['amount'].rolling('6H').mean()
avgVol3h = ticker['amount'].rolling('3H').mean()
avgVol1h = ticker['amount'].rolling('H').mean()
avgVol15m = ticker['amount'].rolling('15min').mean()
avgVol5m = ticker['amount'].rolling('5min').mean()
avgVol1m = ticker['amount'].rolling('1min').mean()

In [ ]:
# ticker['amount'] / 
plt.figure()
plt.plot((ticker['amount'] / avgVol3h))
plt.plot((ticker['amount'] / avgVol1h))
plt.plot((ticker['amount'] / avgVol15m), alpha=0.5)
plt.plot((ticker['amount'] / avgVol5m), alpha=0.25)
plt.plot((ticker['amount'] / avgVol1m), alpha=0.1)
plt.show()

In [ ]:
ticker['averageVol6H'] = ticker['amount'].rolling('6H').mean()
ticker['averageVol3H'] = ticker['amount'].rolling('3H').mean()
ticker['averageVol1H'] = ticker['amount'].rolling('1H').mean()
ticker['averageVol15m'] = ticker['amount'].rolling('15min').mean()
ticker['averageVol5m'] = ticker['amount'].rolling('5min').mean()
ticker['averageVol1m'] = ticker['amount'].rolling('1min').mean()

In [ ]:
ticker['RelVol6H'] = ticker['amount'] / ticker['averageVol6H']
ticker['RelVol3H'] = ticker['amount'] / ticker['averageVol3H']
ticker['RelVol1H'] = ticker['amount'] / ticker['averageVol1H']
ticker['RelVol15m'] = ticker['amount'] / ticker['averageVol15m']
ticker['RelVol5m'] = ticker['amount'] / ticker['averageVol5m']
ticker['RelVol1m'] = ticker['amount'] / ticker['averageVol1m']

ticker.drop(columns=['averageVol6H', 'averageVol3H', 'averageVol1H', 'averageVol15m', 'averageVol5m', 'averageVol1m'], inplace=True)

In [ ]:
plt.plot(((ticker['averageVol1H'] - ticker['averageVol1m']))[:500000])
plt.plot(((ticker['averageVol1H'] - ticker['averageVol5m']))[:500000])

In [ ]:
tws = ['6H', '3H', '1H', '15m', '5m', '1m']
i=0
plt.figure()
for tw in tws:
    print(tw)
    plt.plot(ticker['RelVol'+tw][1000000:1200000], alpha=0.2)
    i += 1
plt.legend(tws)
plt.show()

### **Tick period - average time between mid price change**

In [ ]:
import pandas as pd
import gc
# ticker = pd.read_csv('/kaggle/input/hw1parquets/ticker.csv/ticker.csv')
# ticker.head()

In [ ]:
# ! брать время по remote_ts2
ticker['mid_price'] = (ticker['bid_price'] + ticker['ask_price']) / 2
ticker['price_diff'] = ticker['mid_price'].diff()

ticker['group_id'] = (ticker['price_diff'] != 0).cumsum()
# ticker['group_id_up'] = (ticker['price_diff'] > 0).cumsum()
# ticker['group_id_down'] = (ticker['price_diff'] < 0).cumsum()

ticker['expected_group'] = ticker['group_id'] - 1
# ticker['expected_group_up'] = ticker['group_id_up'] - 1
# ticker['expected_group_down'] = ticker['group_id_down'] - 1

In [ ]:
ticker.head()

In [ ]:
tc = pd.DataFrame(ticker.groupby(['group_id'])['remote_ts'].max()) 
tc.rename(columns={"remote_ts": "change_time"}, inplace=True)

In [ ]:
ticker = pd.merge(ticker, tc, left_on='expected_group', right_on='group_id')
ticker.head()

In [ ]:
ticker['time_since_price_change'] = ticker['remote_ts'] - ticker['change_time']
ticker.head()

In [ ]:
ticker.columns

In [ ]:
ticker = ticker[['remote_ts', 'time_since_price_change', 'bid_price', 'bid_amount', 'ask_price', 'ask_amount', 'spread', 'amount', 'RelVol6H', 'RelVol3H', 'RelVol1H',
       'RelVol15m', 'RelVol5m', 'RelVol1m']]
ticker.head()

In [ ]:
gc.collect()

In [ ]:
# print(ticker.memory_usage(deep=True).sum() / 1024**2)
# print(ticker.dtypes)
# float_cols = ticker.select_dtypes(include='float').columns.tolist()
# ticker[float_cols]=ticker[float_cols].astype('float32')
# print(ticker.memory_usage(deep=True).sum() / 1024**2)

In [ ]:
ticker.head()

In [ ]:
ticker.to_parquet('tickerFeatures.parquet')

# считаем все фичи на буке, тикере, трейдах. Мерджим. Считаем. Если есть прирост то добавляем остальные рынки и все хорошо. 

In [ ]:
book.dtypes
float_cols = book.select_dtypes(include='float').columns.tolist()
book[float_cols]=book[float_cols].astype('float32')
book[['hour', 'part_of_day']]=book[['hour', 'part_of_day']].astype('int16')

In [ ]:
book.dtypes

In [ ]:
book[['buy_amount', 'sell_amount', 'total_amount']] = book[['buy_amount', 'sell_amount', 'total_amount']].astype('float32')

In [ ]:
book.to_parquet('bookFeatures3.parquet')

# **Load**

In [ ]:
import pandas as pd
import numpy as np
tickerF = pd.read_parquet('/kaggle/input/hw1parquets/tickerFeatures.parquet')
bookF = pd.read_parquet('/kaggle/input/hw1parquets/bookFeatures3.parquet')

bookF.drop(columns=[
    'asks[0].amount','bids[0].amount',
#     'AskExecRate500ms', 'AskExecRate1500ms', 'AskExecRate2500ms', 'AskExecRate5000ms', 
#     'BidExecRate500ms', 'BidExecRate1500ms', 'BidExecRate2500ms', 'BidExecRate5000ms', 
    'askExecSum500ms', 'askExecSum1500ms', 'askExecSum2500ms', 'askExecSum5000ms',
    'bidExecSum500ms', 'bidExecSum1500ms', 'bidExecSum2500ms', 'bidExecSum5000ms',
    'timeAskExecute500ms','timeAskExecute1500ms','timeAskExecute2500ms','timeAskExecute5000ms',
                   ], inplace=True)

In [ ]:
bookF.dropna(inplace=True)

In [ ]:
bookF.iloc[1000000:1000010].head(3)

In [ ]:
tickerF['remote_ts'] = tickerF['remote_ts']  * 1000000

In [ ]:
tickerF.drop(columns=['bid_price', 'bid_amount', 'ask_price', 'ask_amount', 'amount'], inplace=True)

print(tickerF.memory_usage(deep=True).sum() / 1024**2)
float_cols = tickerF.select_dtypes(include='float').columns.tolist()
tickerF[float_cols]=tickerF[float_cols].astype('float16')
print(tickerF.memory_usage(deep=True).sum() / 1024**2)

# .print(bookF.memory_usage(deep=True).sum() / 1024**2)
# float_cols = bookF.select_dtypes(include='float').columns.tolist()
# bookF[float_cols]=bookF[float_cols].astype('float16')
# print(bookF.memory_usage(deep=True).sum() / 1024**2)

import gc
bookF.reset_index(inplace=True)
bookF.drop(columns=['dt'], inplace=True)
gc.collect()
bookF.sort_values('timestamp', inplace=True)
tickerF.sort_values('remote_ts', inplace=True)
gc.collect()
bookF.fillna(0, inplace=True)
tickerF.fillna(0, inplace=True)
bookF[['hour', 'part_of_day']] = bookF[['hour', 'part_of_day']].astype('int8')
tickerF['time_since_price_change'] = tickerF['time_since_price_change'].astype('int16') 
gc.collect()

# book.drop(columns=['asks[0].amount', 'bids[0].amount'], inplace=True)



In [ ]:
final3 = pd.merge_asof(tickerF, bookF, left_on='remote_ts', right_on='timestamp')
del tickerF
del bookF
gc.collect()
final3.fillna(0, inplace=True)
final3[['hour', 'part_of_day']] = final3[['hour', 'part_of_day']].astype('int8')
final3.drop(columns=['timestamp'], inplace=True)
final3.head()

In [ ]:
import pandas as pd
import numpy as np
tf = pd.read_parquet('/kaggle/input/hw1parquets/tradeFeatures.parquet')
import gc
tf.rename(columns={"local_ts_x": "local_ts"}, inplace=True)
good_cols = ['local_ts', 'PastRet100ms_x',
'PastRet250ms_x', 'PastRet500ms_x', 'PastRet1000ms_x', 'PastRet2000ms_x',
'PastRet250ms_y', 'PastRet500ms_y', 'PastRet1000ms_y', 'PastRet2000ms_y',
'Div5S', 'Div9S', 'Div19S', 'Div38S', 'Div75S', 'Div150S', 'Div300S', 'Div600S'
]
tf = tf[good_cols]
gc.collect()
float_cols = tf.select_dtypes(include='float').columns.tolist()
tf[float_cols]=tf[float_cols].astype('float16')
ob = pd.read_parquet('/kaggle/input/hw1parquets/ob_features.parquet')
ob_f = ['timestamp', '100msTradeImb_x',
       '250msTradeImb_x', '500msTradeImb_x', '1000msTradeImb_x',
       '2000msTradeImb_x',
        '100msTradeImb_y',
       '250msTradeImb_y', '500msTradeImb_y', '1000msTradeImb_y',
       '2000msTradeImb_y']
ob = ob[ob_f]
gc.collect()
print(len(ob))
ob.dropna(inplace=True)
tf.dropna(inplace=True)
print(len(ob))
target = pd.read_parquet('/kaggle/input/hw1parquets/target.parquet')
target['target'] = target['target'].astype('int8')
# ob_float_cols = ob.select_dtypes(include='float').columns.tolist()
ob_f = ob_f[1:]
ob[ob_f]=ob[ob_f].astype('float16')
n = pd.merge_asof(target, ob, left_on='local_ts', right_on='timestamp')
n.dropna(inplace=True)
del ob
del target
gc.collect()

final = pd.merge_asof(n, tf, on='local_ts')
del n
del tf
gc.collect()
# final.drop(columns=['local_ts', 'timestamp'], inplace=True)
final.head()

In [ ]:
final.drop(columns=['timestamp'], inplace=True)
final = pd.merge_asof(final, final3, left_on='local_ts', right_on='remote_ts')

del final3
gc.collect()

In [ ]:
final_columns = final.columns[1:]

In [ ]:
final.drop(columns=['local_ts', 'remote_ts'], inplace=True)

In [ ]:
final_columns = final.columns[1:]

In [ ]:
gc.collect()

In [ ]:
# final.to_parquet('newHw13.parquet')

# Read features

In [ ]:
import numpy as np
import pandas as pd
final =  pd.read_parquet('/kaggle/input/hw1parquets/hw13new.parquet')

In [ ]:
high_float_cols = [
        'AskExecRate500ms', 'BidExecRate500ms', 'timeBidExecute500ms',
       'AskExecRate1500ms', 'BidExecRate1500ms', 'timeBidExecute1500ms',
       'AskExecRate2500ms', 'BidExecRate2500ms', 'timeBidExecute2500ms',
       'AskExecRate5000ms', 'BidExecRate5000ms', 'timeBidExecute5000ms',]

float_cols = ['100msTradeImb_x', '250msTradeImb_x', '500msTradeImb_x',
       '1000msTradeImb_x', '2000msTradeImb_x', '100msTradeImb_y',
       '250msTradeImb_y', '500msTradeImb_y', '1000msTradeImb_y',
       '2000msTradeImb_y', 'PastRet100ms_x', 'PastRet250ms_x',
       'PastRet500ms_x', 'PastRet1000ms_x', 'PastRet2000ms_x',
       'PastRet250ms_y', 'PastRet500ms_y', 'PastRet1000ms_y',
       'PastRet2000ms_y', 'Div5S', 'Div9S', 'Div19S', 'Div38S', 'Div75S',
       'Div150S', 'Div300S', 'Div600S',  'spread',
       'RelVol6H', 'RelVol3H', 'RelVol1H', 'RelVol15m', 'RelVol5m', 'RelVol1m',
       'NewAskDiff', 'NewBidDiff', 'buy_amount', 'sell_amount', 'total_amount',
       
       '250msStd', '500msStd', '1000msStd', '2500msStd']

int_cols = ['time_since_price_change', 'hour', 'part_of_day']

final[float_cols] = final[float_cols].astype('float16')

# final.to_parquet('hw13new.parquet')


In [ ]:
final.head(10000).tail(10)

In [ ]:
import gc
final_columns = final.columns[1:]
y = final['target']
final.drop(columns=['target'], inplace=True)
gc.collect()

In [ ]:
import gc
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

model = LogisticRegression()
scaler = StandardScaler(copy=False)

X_train, X_test, y_train, y_test = train_test_split(final, y, test_size=0.5, random_state=42, shuffle=False)
l = len(final)
del final
del y

# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.6, random_state=42, shuffle=False)


# print(len(X_train) , len(X_val) , len(X_test) )



In [ ]:
X_train[np.isinf(X_train)] = np.iinfo(np.int16).max
X_test[np.isinf(X_test)] = np.iinfo(np.int16).max
gc.collect()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)
# X_val = scaler.transform(X_val)

In [ ]:
gc.collect()

In [ ]:
model = LogisticRegression(penalty='l2',max_iter=25, C=0.05)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

import numpy as np
from sklearn.metrics import confusion_matrix

def metric(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    weight_matrix = np.array([
        [1.9,     0,    -2],
        [-0.3,    0,    -0.3],
        [-2,      0,     1.9]
    ])
    
    hit_matrix = conf_matrix * weight_matrix
    hit_matrix_sum = np.sum(hit_matrix)
    action_count = np.sum(conf_matrix[:, 0]) + np.sum(conf_matrix[:, 2])
    if action_count == 0:
        return 0
    else:
        return hit_matrix_sum / np.sqrt(action_count)
    
res_train = metric(y_train, y_pred_train)
# res_val = metric(y_val, y_pred_val)
res_test = metric(y_test, y_pred_test)
print(res_train, res_test)

In [ ]:
gc.collect()

In [ ]:
# get the feature names and coefficients
import matplotlib.pyplot as plt
feature_names = final_columns
coefficients = model.coef_[2]

# create a dictionary with feature names and coefficients
feature_importances = dict(zip(feature_names, coefficients))

# sort the dictionary by absolute value of coefficients
sorted_feature_importances = sorted(feature_importances.items(), key=lambda x: abs(x[1]), reverse=True)

# create bar plot of feature importances
fig, ax = plt.subplots(figsize=(10,6))
ax.bar([f[0] for f in sorted_feature_importances], [f[1] for f in sorted_feature_importances])
ax.set_xticklabels([f[0] for f in sorted_feature_importances], rotation=90, fontsize=11)
ax.set_ylabel('Feature Importance', fontsize=14)
plt.show()

In [ ]:
sorted_feature_importances[:10]

# HW4 

# **Postprocessing**

Хотим подобрать такие пороги для -1 и 1 прогнозов, чтобы максимизировать метрику относительно просто аргмакса.

Базовая идея: для каждого класса возьмем сетку порогов, переберем все комбинации и посмотрим какая тройка порогов лучше всего максимизирует метрику. 
* Минусы: не учитываем вероятности других классов, помимо того который рассматриваем, может быть долго.
* Плюсы: дешево и сердито, можно использовать как начальный бейзлайн 

Вторая идея: Обучим решающее дерево на вероятностных аутпутах и попробуем так подобрать оптимальный постпроцессинг, который будет сразу учитывать вероятность каждого класса, а не только число конкретного
* Минусы: можно случайно оверфитнуться
* Плюсы: учитываем взаимосвязи вероятностей, можно подобрать более точные пороги

In [ ]:
y_train_probs = model.predict_proba(X_train)
y_test_probs = model.predict_proba(X_test)

In [ ]:
gc.collect()

In [ ]:
%%time 
y_pred_train[y_train_probs[:, 2] >= tp] = 1
y_pred_train[(y_train_probs[:, 0] >= tn)] = -1
y_pred_train[(y_train_probs[:, 0] < tn) & (y_train_probs[:, 2] < tp)] = 0
metric_train = metric(y_train, y_pred_train) 

In [ ]:
gc.collect()

In [ ]:
tsNeg = np.arange(0.3, 0.45, 0.015)
tsPos = np.arange(0.3, 0.45, 0.015)

best_m = 0

for tn in tsNeg:
    for tp in tsPos:
        gc.collect()
        y_pred_train = np.zeros(y_train_probs.shape[0])
        y_pred_test = np.zeros(y_test_probs.shape[0])
        print("ths: ", tn, tp)
        y_pred_train[(y_train_probs[:, 2] >= tp)] = 1
        y_pred_train[(y_train_probs[:, 2] < tp) & (y_train_probs[:, 0] >= tn)] = -1
        y_pred_train[(y_train_probs[:, 0] < tn) & (y_train_probs[:, 2] < tp)] = 0
        metric_train = metric(y_train, y_pred_train) 
        if metric_train > best_m:
            best_m = metric_train
            y_pred_test[(y_test_probs[:, 2] >= tp)] = 1
            y_pred_test[(y_test_probs[:, 2] < tp) & (y_test_probs[:, 0] >= tn)] = -1
            y_pred_test[(y_test_probs[:, 0] < tn) & (y_test_probs[:, 2] < tp)] = 0
            metric_test = metric(y_test, y_pred_test)
            print("Metric train and test: ", metric_train, metric_test)
            
            
        

In [ ]:
print(best_m)

Заметим что самые лучшие пороги это ~0.36, 0.39, и в их окрестонтсях надо более точно оптимизировать. 

In [ ]:
# TODO NEXT
tsNeg = np.arange(0.34, 0.39, 0.01)
tsPos = np.arange(0.34, 0.39, 0.01)
best_m = 0
for tn in tsNeg:
    for tp in tsPos:
        gc.collect()
        y_pred_train = np.zeros(y_train_probs.shape[0])
        y_pred_test = np.zeros(y_test_probs.shape[0])
        y_pred_train[(y_train_probs[:, 2] >= tp)] = 1
        y_pred_train[(y_train_probs[:, 2] < tp) & (y_train_probs[:, 0] >= tn)] = -1
        y_pred_train[(y_train_probs[:, 0] < tn) & (y_train_probs[:, 2] < tp)] = 0
        metric_train = metric(y_train, y_pred_train) 
        if metric_train > best_m:
            print("ths: ", tn, tp)
            best_m = metric_train
            y_pred_test[(y_test_probs[:, 2] >= tp)] = 1
            y_pred_test[(y_test_probs[:, 2] < tp) & (y_test_probs[:, 0] >= tn)] = -1
            y_pred_test[(y_test_probs[:, 0] < tn) & (y_test_probs[:, 2] < tp)] = 0
            metric_test = metric(y_test, y_pred_test)
            print("Metric train and test: ", metric_train, metric_test)

Возьму тот порог который дает самый нормальный результат на тесте: 0.34, 0.35

### Более интересно попробовать подход с решающим деревом на вероятностях

In [ ]:
# Пробуем подход с решающим деревом.

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=4)

dt.fit(y_train_probs, y_train)

y_train_pred_dt = dt.predict(y_train_probs)
y_test_pred_dt = dt.predict(y_test_probs)

dt_metric_train = metric(y_train, y_train_pred_dt)
dt_metric_test = metric(y_test, y_test_pred_dt)
print("Tree postprocessing: ", dt_metric_train, dt_metric_test)

Получилась фигня